In [3]:
import numpy as np
import pandas as pd
import seaborn as sns
from matplotlib import pyplot as plt

from statsmodels.tsa.api import ExponentialSmoothing, SimpleExpSmoothing, Holt
from sklearn.linear_model import LinearRegression

import warnings
warnings.filterwarnings('ignore')

In [4]:
df = pd.read_csv("monthly_csv.csv")
df.head()

,Date,Price
0,1950-01,34.73
1,1950-02,34.73
2,1950-03,34.73
3,1950-04,34.73
4,1950-05,34.73


In [5]:
df.shape


(847, 2)

In [ ]:
#eda

In [6]:
print(f"Date range of gold prices available from -{df.loc[:,'Date'][0]}to {df.loc[:,'Date'][len(df)-1]}")

Date range of gold prices available from -1950-01to 2020-07


In [ ]:
date = pd.date_range (state = '1/1/1950')